In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive/My Drive/ImageCaptioningPRO/data

/content/drive/My Drive/ImageCaptioningPRO/data


In [3]:
pwd

'/content/drive/My Drive/ImageCaptioningPRO/data'

In [0]:
import os 
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
from os import listdir
import tensorflow.contrib.keras as keras
from PIL import Image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding, Dropout
from tensorflow.python.keras.applications import VGG16
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.backend import manual_variable_initialization
manual_variable_initialization(True)

In [0]:
def load_imagesdict(directory):
  images = dict()
  count = 0
  for name in listdir(directory):
    if count%500 == 0:
      print(count)
      #print("shape",image.shape[0],image.shape[1],image.shape[2])
    filename = directory + '/' + name
    image = load_img(filename,target_size = (224,224))
    #convert pixels to numpy array
    image = img_to_array(image)
    #reshape for model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    image_id = name.split(".")[0]
    images[image_id] = image
    count = count + 1
  return images

In [0]:
def load_doc(filename):
  file = open(filename,'r')
  text = file.read()
  file.close()
  return text


def load_descriptions(doc):
  captions = dict()
  count=0
  for line in doc.split('\n'):
    tokens = line.split()
    if len(line)<2:
      continue
    count = count+1
    image_id, image_desc = tokens[0], tokens[1:]
    image_id = image_id.split('.')[0]
    image_desc = ' '.join(image_desc)
    if image_id not in captions:
      captions[image_id] = image_desc
  return captions

      
def loadcaptionsdict(filename):
  doc = load_doc(filename)
  descriptions = load_descriptions(doc)
  return descriptions


In [0]:
def getimgandcaptionlist(images,descriptions):
  imgids = list(images.keys())
  Images = []
  captions = []
  for _ in range(len(imgids)):
    Images.append(images[imgids[_]])
    captions.append(descriptions[imgids[_]])
  return Images,captions

In [0]:
def mark_captions(captions):
  mark_start = 'ssss '
  mark_end = ' eeee'
  captions_marked = [mark_start + caption + mark_end for caption in captions]
  return captions_marked

In [0]:
def sentences_to_list(captions_marked):
  listtowords = []
  for _ in captions_marked:
    w = _.split()
    if '.' in w:
      w.remove('.')
    listtowords.append(w)
  return listtowords

In [0]:
def get_unique_wordsdict(captions_marked_list):
  unique_words = []
  for i in range(len(captions_marked_list)):
    for j in range(len(captions_marked_list[i])):
      unique_words.append(captions_marked_list[i][j])
  unique_words = list(set(unique_words))
  inttoword = dict(list(enumerate(unique_words)))
  return inttoword

In [0]:
def invert_dict(inttoword):
  return dict([(v,k) for k,v in inttoword.items()])

In [0]:
def word2int(caps,unique_words_dict):
  for i in range(len(caps)):
    for j in range(len(caps[i])):
      caps[i][j] = unique_words_dict[caps[i][j]]
  return caps

In [0]:
def VGGModel():
  image_model = VGG16(include_top=True, weights = 'imagenet')
  transfer_layer = image_model.get_layer('fc2')
  image_model_transfer = Model(inputs = image_model.inputs, outputs = transfer_layer.output)
  return image_model_transfer, transfer_layer.output

In [0]:
def decoder(state_size,embedding_size,maxnumwords,transfer_layer_output):
  transfer_value_size = K.int_shape(transfer_layer_output)[1]
  transfer_value_input = Input(shape=(transfer_value_size,),name = 'transfer_value_input')
  decoder_transfer_map = Dense(state_size,activation = 'tanh',name = 'decoder_transfer_map')
  decoder_input = Input(shape=(None,),name = 'decoder_input')
  decoder_embedding = Embedding(input_dim=num_words,output_dim=embedding_size,name = 'decoder_embedding')
  dropout_1 = Dropout(0.2)
  decoder_gru = GRU(state_size,name='decoder_gru',return_sequences=True)
  decoder_dense = Dense(num_words,activation='linear',name='decoder_output')
  
  initial_state = decoder_transfer_map(transfer_value_input)
  initial_state = dropout_1(initial_state)
  net = decoder_input
  net = decoder_embedding(net)
  net = decoder_gru(net, initial_state=initial_state)
  decoder_output = decoder_dense(net)
  decoder_model = Model(inputs=[transfer_value_input, decoder_input],outputs=[decoder_output])
  
  return decoder_model

In [0]:
def sparse_cross_entropy(y_true, y_pred):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred)
    loss_mean = tf.reduce_mean(loss)
    return loss_mean

In [0]:
def get_imgsignature(image_model_transfer,images,num):
  trans_val = []
  for _ in range(num):
    trans_val.append(image_model_transfer.predict(images[_]))
  print("Image signatures obtained...")
  return trans_val

In [0]:
def batch_generator(batch_size):
  while True:
        idx = np.random.randint(len(Images),size=batch_size)
        
        transf_values = np.array([transfer_values[_] for _ in idx])
        tokens = [caps_markedwords[_] for _ in idx]

        num_tokens = [len(t) for t in tokens]
        
        max_tokens = np.max(num_tokens)
        tokens_padded = pad_sequences(tokens,
                                      maxlen=max_tokens,
                                      padding='post',
                                      truncating='post')
        
        decoder_input_data = tokens_padded[:, 0:-1]
        decoder_output_data = tokens_padded[:, 1:]

        x_data = \
        {
            'decoder_input': decoder_input_data,
            'transfer_value_input': transf_values
        }

        y_data = \
        {
            'decoder_output': decoder_output_data
        }
        
        yield (x_data, y_data)

In [0]:
def generate_caption(image_model_transfer,image_path,unique_words_dict,max_tokens=30):
  token_end = unique_words_dict['eeee']
  image = load_img(image_path, target_size=(224, 224))
  a = image
    
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  
  transfer_values = image_model_transfer.predict(image)
  
  shape = (1, max_tokens)
    
  decoder_input_data = np.zeros(shape=shape, dtype=np.int)
  token_int = unique_words_dict["ssss"]
  output_text = ''
  count_tokens = 0
    
  while token_int != token_end and count_tokens < max_tokens:
       
      decoder_input_data[0, count_tokens] = token_int
      
      x_data = \
      {
          'transfer_value_input': transfer_values,
          'decoder_input': decoder_input_data
      }
      decoder_output = decoder_model.predict(x_data)
      
      token_onehot = decoder_output[0, count_tokens, :]
        # Convert to an integer-token.
      token_int = np.argmax(token_onehot)
      sampled_word = inttoword[token_int]
        
        # Append the word to the output-text.
      output_text += " " + sampled_word

        # Increment the token-counter.
      count_tokens += 1
  output_tokens = decoder_input_data[0]
    
  print("Image Path" + image_path)
  print("Predicted caption:")
  print(output_text)
  return output_text

In [0]:
directory = 'Flicker8k_Dataset'
filename = 'Flickr8k.token.txt'

In [17]:
imageDict = load_imagesdict(directory)
print(len(imageDict))

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8091


In [0]:
state_size = 512
embedding_size = 128
batch_size = 32

In [0]:
descriptions = loadcaptionsdict(filename)
Images,captions = getimgandcaptionlist(imageDict,descriptions)

captions_marked = mark_captions(captions)

caps_markedwords = sentences_to_list(captions_marked)
inttoword = get_unique_wordsdict(caps_markedwords)
wordtoint = invert_dict(inttoword)

num_words = len(inttoword.keys())

capsintencoded = word2int(caps_markedwords, wordtoint)

tokens_caps = [capsintencoded[_] for _ in range(len(capsintencoded))]
num_tok = [len(t) for t in tokens_caps]

max_toks = np.max(num_tok)
tokens_padded = pad_sequences(tokens_caps,maxlen=max_toks,padding='post',truncating='post')

dec_inp = tokens_padded[:,0:-1]
dec_out = tokens_padded[:,1:]

num_images = len(Images)
steps_per_epoch = int(num_images/batch_size)


In [31]:
import pickle

pickle_out = open("wordtoint.pickle","wb")
pickle.dump(wordtoint,pickle_out)
pickle_out.close()

pickle_out = open("inttoword.pickle","wb")
pickle.dump(inttoword,pickle_out)
pickle_out.close()

print(inttoword.keys())
print(wordtoint.keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [0]:
import pickle

pickle_out = open("wordtoint.pickle","rb")
wordtoint = pickle.load(pickle_out)
pickle_out.close()

pickle_out = open("inttoword.pickle","rb")
inttoword = pickle.load(pickle_out)
pickle_out.close()

In [37]:
print(inttoword.keys())
print(wordtoint.keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [45]:
model,transfer_layer_output = VGGModel()
decoder_model = decoder(state_size,embedding_size,num_words,transfer_layer_output)
optimizer = RMSprop(lr=1e-3)
#manually create a placeholder variable for the decoder's output
decoder_target = tf.placeholder(dtype='int32', shape=(None,None))
decoder_model.compile(optimizer=optimizer, loss=sparse_cross_entropy,target_tensors=[decoder_target])

transfer_values = get_imgsignature(model,Images,num_images)
transfer_values = np.array(transfer_values).reshape(num_images,4096)
np.savetxt('/content/drive/My Drive/ImageCaptioningPRO/data/vgg_transfer_values.txt',transfer_values,fmt="%d")


Image signatures obtained...


In [46]:
transfer_values[20]

array([5.1788955, 0.       , 1.5111248, ..., 0.       , 5.4544463,
       0.       ], dtype=float32)

In [49]:
np.savetxt('/content/drive/My Drive/ImageCaptioningPRO/data/vgg_transfer_values.npy',transfer_values,fmt="%d")
transfer_values1 = np.loadtxt('vgg_transfer_values.npy')
transfer_values1[20]

array([5., 0., 1., ..., 0., 5., 0.])

In [0]:
path_checkpoint = '/content/drive/My Drive/ImageCaptioningPRO/data/gru1_30epoch_plus.keras'
callback_checkpoint = ModelCheckpoint(filepath = path_checkpoint,verbose=1,save_weights_only=True)
callback_tensorboard = TensorBoard(log_dir='./22_logs/',histogram_freq=0,write_graph=False)
callbacks = [callback_checkpoint, callback_tensorboard]
generator = batch_generator(batch_size)

In [60]:
decoder_model.fit_generator(generator=generator,steps_per_epoch=steps_per_epoch,epochs=30,callbacks=callbacks)

Epoch 1/30
251/252 [============================>.] - ETA: 0s - loss: 2.0465
Epoch 00001: saving model to /content/drive/My Drive/ImageCaptioningPRO/data/gru1_30epoch_plus.keras
252/252 [==============================] - 18s 70ms/step - loss: 2.0457
Epoch 2/30
251/252 [============================>.] - ETA: 0s - loss: 1.8759
Epoch 00002: saving model to /content/drive/My Drive/ImageCaptioningPRO/data/gru1_30epoch_plus.keras
252/252 [==============================] - 18s 73ms/step - loss: 1.8763
Epoch 3/30
251/252 [============================>.] - ETA: 0s - loss: 1.7568
Epoch 00003: saving model to /content/drive/My Drive/ImageCaptioningPRO/data/gru1_30epoch_plus.keras
252/252 [==============================] - 18s 72ms/step - loss: 1.7553
Epoch 4/30
251/252 [============================>.] - ETA: 0s - loss: 1.6440
Epoch 00004: saving model to /content/drive/My Drive/ImageCaptioningPRO/data/gru1_30epoch_plus.keras
252/252 [==============================] - 18s 72ms/step - loss: 1.6428


In [71]:
img = 't10.jpg'
generate_caption(model,img,wordtoint,max_tokens=30)

Image Patht10.jpg
Predicted caption:
 A man in a blue shirt is running down the field eeee


' A man in a blue shirt is running down the field eeee'